In [1]:
# Branch constants
old_branch = 'low-py'
branch = 'compress'

In [2]:
# Load files
from plotting import load_and_parse
load_old_branch = False
if load_old_branch:
    load_and_parse(old_branch)
load_and_parse(branch)

Total runtime: 3 days, 3:37:01
Total runtime: 19:44:22


In [3]:
# Read files
from plotting import fetch_times
old_global_times = fetch_times(old_branch)
global_times = fetch_times(branch)
old_sectioned_times = None
sectioned_times = None
try:
    old_sectioned_times = fetch_times(old_branch, True)
except FileNotFoundError:
    print("No old sectioned runs found.")
try:
    sectioned_times = fetch_times(branch, True)
except FileNotFoundError:
    print("No sectioned runs found.")

In [4]:
# Plot general runtimes
from plotting import get_time_plots
inter = 'raw'
for robu in 'basic', 'robust':
    version = (inter, robu)
    for k in global_times[version]['K'].unique():
        for d in 1, 2, 5, 10:
            get_time_plots(
                global_times[version], branch,
                old_global_times[version], old_branch,
                "runtime", f"{version[0]} {version[1]} K={k} d={d}",
                k=k, degree=d
            )

<Figure size 1382.4x758.88 with 0 Axes>

In [ ]:
# Plot section runtimes
if sectioned_times is not None and old_sectioned_times is not None:
    from plotting import get_time_plots
    version = ('raw', 'basic')
    section = "time_index"
    complement = False
    from pandas import DataFrame
    sectioned_times: DataFrame
    old_sectioned_times: DataFrame
    if complement:
        section_times = sectioned_times.groupby(['run', 'algorithm', 'n', 'K', 'degree'], as_index=False).sum()
        old_section_times = old_sectioned_times.groupby(['run', 'algorithm', 'n', 'K', 'degree'], as_index=False).sum()
    else:
        section_times = sectioned_times[version][sectioned_times[version]['section'] == section]
        old_section_times = old_sectioned_times[version][old_sectioned_times[version]['section'] == section]
    for k in sectioned_times[version]['K'].unique():
        for d in 1, 2, 5, 10:
            get_time_plots(
                section_times, branch,
                old_section_times, old_branch,
                "hashing", f"{version[0]} {version[1]} K={k} d={d}",
                k=k, degree=d
            )

In [5]:
# Tabulate speedups
from plotting import get_speedup_tables
inter = 'raw'
for robu in 'basic', 'robust':
    version = (inter, robu)
    master_global_times = fetch_times(old_branch)
    for k in global_times[version]['K'].unique():
        for d in 1, 2, 5, 10:
            get_speedup_tables(
                global_times[version], branch, master_global_times[version],
                "time", k=k, degree=d, dest=f"{version[0]}_{version[1]}_K{k}_d{d}"
            )

In [6]:
# Plot profiles
if sectioned_times is not None:
    from plotting import get_profiling, CS_ALGOS
    from numpy import partition
    inter = 'raw'
    minmax = lambda data: (data.min(), partition(data.unique(), -2)[-2])
    minmax2 = lambda data: (partition(data.unique(), 1)[1], partition(data.unique(), -2)[-2])
    for robu in 'basic', 'robust':
        version = (inter, robu)
        for cs in CS_ALGOS:
            for k in minmax(sectioned_times[version]['K']):
                for d in 2, 10:
                    for n in minmax2(sectioned_times[version]['n']):
                        get_profiling(
                            sectioned_times[version], branch,
                            f"{version[0]} {version[1]} {cs} n={n} K={k} d={d}",
                            cs, n, k, d
                        )

Skipping: naive 7502 203 2
Skipping: naive 7502 203 10


<Figure size 1382.4x758.88 with 0 Axes>